In [2]:
#import libraries
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import *

# Load Files

In [3]:

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("MyApp") \
    .getOrCreate()


# create a dataframe

In [ ]:

file_path = r"D:\Data Analysis\Python\stat\Project\googleplaystore.csv"  
df=spark.read.csv(file_path,header=True,inferSchema=True)
df.show()
#df=spark.read.load(file_path,format='csv',sep=',',header=True,escape='"')

# Count

In [6]:
df.count()

10841

# display first row

In [7]:
df.show(1)

+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Size|Installs|Type|Price|Content Rating|      Genres|   Last Updated|Current Ver| Android Ver|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 19M| 10,000+|Free|    0|      Everyone|Art & Design|January 7, 2018|      1.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+----+--------+----+-----+--------------+------------+---------------+-----------+------------+
only showing top 1 row



# check Schema

In [8]:

df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Content Rating: string (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Last Updated: string (nullable = true)
 |-- Current Ver: string (nullable = true)
 |-- Android Ver: string (nullable = true)



# Data Cleaning


Drop the columns which are not important

In [9]:
df=df.drop("size","Content Rating","Last Updated","Android Ver","Current Ver")

In [10]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|Current Ver| Android Ver|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|      1.0.0|4.0.3 and up|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|      2.0.0|4.0.3 and up|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+------------+
only showing top 2 rows



In [11]:
df=df.drop("Android Ver")

In [12]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|Current Ver|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|      1.0.0|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|      2.0.0|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+-----------+
only showing top 2 rows



In [13]:
df=df.drop("Current Ver")

In [14]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159| 10,000+|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|500,000+|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [15]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Reviews: string (nullable = true)
 |-- Installs: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Genres: string (nullable = true)



Change the type of columns, remove '+' from Installs column etc

In [16]:
from pyspark.sql.functions import regexp_replace,col
df=df.withColumn("Reviews",col("Reviews").cast(IntegerType()))\
.withColumn("Installs",regexp_replace(col("Installs"),"[^0-9]",""))\
.withColumn("Installs",col("Installs").cast(IntegerType()))\
.withColumn("Price",regexp_replace(col("Price"),"[$]",""))\
.withColumn("Price",col("Price").cast(IntegerType()))

In [17]:
df.show(2)

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
only showing top 2 rows



In [66]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col

# Convert column to FloatType (or DoubleType)
df = df.withColumn("Rating", col("Rating").cast(FloatType()))

In [49]:
# Replace "NaN" (string) with None (null values)
from pyspark.sql.functions import isnan
df = df.withColumn("Rating", when(isnan(col("Rating")),0).otherwise(col("Rating")))
#|df = df.fillna({"Rating": 0})

In [ ]:
df = df.withColumn("Rating", when(col("Rating") == "NaN", np.nan).otherwise(col("Rating")))
df = df.fillna({"Rating": 0})  # Replace null values in "Rating" column with 0

In [ ]:
df.createOrReplaceTempView("apps")

In [67]:
df.printSchema()

root
 |-- App: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Rating: float (nullable = true)
 |-- Reviews: integer (nullable = true)
 |-- Installs: integer (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Genres: string (nullable = true)



# Create a Temperory View of the table

In [19]:
df.createOrReplaceTempView("apps")

# Write Queries

In [22]:
spark.sql("SELECT * from apps").show()

+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|                 App|      Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+--------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|ART_AND_DESIGN|   4.1|  36815| 1000000

# 1. Find out Top 10 reviews given to the apps

In [85]:
spark.sql("SELECT App,sum(Reviews) from apps group by 1 order by 2 desc").show(10)

+--------------------+------------+
|                 App|sum(Reviews)|
+--------------------+------------+
|           Instagram|   266241989|
|  WhatsApp Messenger|   207348304|
|      Clash of Clans|   179558781|
|Messenger – Text ...|   169932272|
|      Subway Surfers|   166331958|
|    Candy Crush Saga|   156993136|
|            Facebook|   156286514|
|         8 Ball Pool|    99386198|
|        Clash Royale|    92530298|
|            Snapchat|    68045010|
+--------------------+------------+
only showing top 10 rows



# 2. Top 10 installs apps and distribution of type(free/Paid)

In [26]:
spark.sql("SELECT App,sum(Installs) from apps group by 1 order by 2 desc").show(10)

+--------------------+-------------+
|                 App|sum(Installs)|
+--------------------+-------------+
|      Subway Surfers|   6000000000|
|           Instagram|   4000000000|
|        Google Drive|   4000000000|
|            Hangouts|   4000000000|
|         Google News|   4000000000|
|       Google Photos|   4000000000|
|    Candy Crush Saga|   3500000000|
|Messenger – Text ...|   3000000000|
|Google Chrome: Fa...|   3000000000|
|Maps - Navigate &...|   3000000000|
+--------------------+-------------+
only showing top 10 rows



In [29]:
spark.sql("SELECT App,Type,sum(Installs) from apps group by 1,2 order by 3 desc").show(10)

+------------------+----+-------------+
|               App|Type|sum(Installs)|
+------------------+----+-------------+
|    Subway Surfers|Free|   6000000000|
|         Instagram|Free|   4000000000|
|      Google Drive|Free|   4000000000|
|          Hangouts|Free|   4000000000|
|     Google Photos|Free|   4000000000|
|       Google News|Free|   4000000000|
|  Candy Crush Saga|Free|   3500000000|
|WhatsApp Messenger|Free|   3000000000|
|             Gmail|Free|   3000000000|
|      Temple Run 2|Free|   3000000000|
+------------------+----+-------------+
only showing top 10 rows



# 3. Categorywise distribution of installed apps

In [28]:
spark.sql("SELECT Category,sum(Installs) from apps group by 1 order by 2 desc").show(truncate=False, n=df.count())

+-------------------+-------------+
|Category           |sum(Installs)|
+-------------------+-------------+
|GAME               |35086024415  |
|COMMUNICATION      |32647276251  |
|PRODUCTIVITY       |14176091369  |
|SOCIAL             |14069867902  |
|TOOLS              |11452771915  |
|FAMILY             |10258263505  |
|PHOTOGRAPHY        |10088247655  |
|NEWS_AND_MAGAZINES |7496317760   |
|TRAVEL_AND_LOCAL   |6868887146   |
|VIDEO_PLAYERS      |6222002720   |
|SHOPPING           |3247848785   |
|ENTERTAINMENT      |2869160000   |
|PERSONALIZATION    |2325494782   |
|BOOKS_AND_REFERENCE|1921469576   |
|SPORTS             |1751174498   |
|HEALTH_AND_FITNESS |1582072512   |
|BUSINESS           |1001914865   |
|FINANCE            |876648734    |
|EDUCATION          |871452000    |
|MAPS_AND_NAVIGATION|719281890    |
|LIFESTYLE          |537643539    |
|WEATHER            |426100520    |
|FOOD_AND_DRINK     |273898751    |
|DATING             |264310807    |
|HOUSE_AND_HOME     |1687124

# 4. Top paid apps

In [30]:
spark.sql("SELECT App,sum(Price) from apps where Type='Paid' group by 1 order by 2 desc").show()

+--------------------+----------+
|                 App|sum(Price)|
+--------------------+----------+
|I'm Rich - Trump ...|       400|
|   I Am Rich Premium|       399|
|  I AM RICH PRO PLUS|       399|
|I'm Rich/Eu sou R...|       399|
|      I am Rich Plus|       399|
|most expensive ap...|       399|
|       I Am Rich Pro|       399|
|  I am rich(premium)|       399|
|           I am Rich|       399|
|          I am Rich!|       399|
|         💎 I'm rich|       399|
|I am rich (Most e...|       399|
|           I am rich|       399|
|         Eu Sou Rico|       394|
|           I Am Rich|       389|
| I am extremely Rich|       379|
|       I am rich VIP|       299|
|        EP Cook Book|       200|
|Vargo Anesthesia ...|       158|
|       cronometra-br|       154|
+--------------------+----------+
only showing top 20 rows



# 5. Top paid rating apps

In [78]:
spark.sql("SELECT * from apps").show(100)

+--------------------+-----------------+------+-------+--------+----+-----+--------------------+
|                 App|         Category|Rating|Reviews|Installs|Type|Price|              Genres|
+--------------------+-----------------+------+-------+--------+----+-----+--------------------+
|Photo Editor & Ca...|   ART_AND_DESIGN|   4.1|    159|   10000|Free|    0|        Art & Design|
| Coloring book moana|   ART_AND_DESIGN|   3.9|    967|  500000|Free|    0|Art & Design;Pret...|
|U Launcher Lite –...|   ART_AND_DESIGN|   4.7|  87510| 5000000|Free|    0|        Art & Design|
|Sketch - Draw & P...|   ART_AND_DESIGN|   4.5| 215644|50000000|Free|    0|        Art & Design|
|Pixel Draw - Numb...|   ART_AND_DESIGN|   4.3|    967|  100000|Free|    0|Art & Design;Crea...|
|Paper flowers ins...|   ART_AND_DESIGN|   4.4|    167|   50000|Free|    0|        Art & Design|
|Smoke Effect Phot...|   ART_AND_DESIGN|   3.8|    178|   50000|Free|    0|        Art & Design|
|    Infinite Painter|   ART_A